# **Tugas Besar B - IF3270 Pembelajaran Mesin**
Authors:
1. 13519096 Girvin Junod
2. 13519116 Jeane Mikha Erwansyah
3. 13519131 Hera Shafira
4. 13519188 Jeremia Axel
---

## Install Libraries

In [77]:
!pip install icecream
!pip install tabulate

## Load libraries

In [78]:
import pandas as pd
import os, subprocess, sys
import json, math, typing, copy
import numpy as np, networkx as nx, matplotlib as plt
from icecream import ic
from tabulate import tabulate

## Enums

In [79]:
class LayerType:
	INPUT = "input"
	HIDDEN = "hidden"
	OUTPUT = "output"

class ActivationFunction:
	SIGMOID = "sigmoid"
	RELU = "relu"
	SOFTMAX = "softmax"
	LINEAR = "linear"

## Utility Functions

In [80]:
class Utils:
	@staticmethod
	def matrix_dimension(mat: typing.List[list]) -> typing.Tuple[int, int]:
		return len(mat), len(mat[0])

	@staticmethod
	def get_layer_type(type: str):
		if type == 'input':
			return LayerType.INPUT
		elif type == 'hidden':
			return LayerType.HIDDEN
		elif type == 'output':
			return LayerType.OUTPUT
	  
	@staticmethod
	def get_activation_func(activation_func: str):
		if activation_func == 'sigmoid':
		  return ActivationFunction.SIGMOID
		elif activation_func == 'linear':
		  return ActivationFunction.LINEAR
		elif activation_func == 'relu':
		  return ActivationFunction.RELU
		elif activation_func == 'softmax':
		  return ActivationFunction.SOFTMAX
		  
	@staticmethod
	def parse_json(filename):
		with open(filename, 'r') as f:
			data = json.load(f)
			return data

	@staticmethod
	def export_json(filename, data):
		with open(filename, 'w') as f:
			f.write(json.dumps(data, indent=2))

	@staticmethod
	def install(package):
		subprocess.check_call([sys.executable, "-m", "pip", "install", ''.join(package)])

In [81]:
class Activations:
  @staticmethod
  def sigmoid(x: np.ndarray):
    return 1/(1+np.exp(-x))
  @staticmethod
  def relu(x):
    return np.maximum(0, x)
  @staticmethod
  def linear(x):
    return x
  @staticmethod
  def softmax(x: np.ndarray):
    e_x = np.exp(x-np.max(x))
    return e_x/e_x.sum(axis=1).reshape(-1,1)
  @staticmethod
  def d_sigmoid(x):
    return Activations.sigmoid(x) * (1 - Activations.sigmoid(x))
  @staticmethod
  def d_linear(x):
    return 1
  @staticmethod
  def d_relu(x):
    return (x>=0)*1
  @staticmethod
  def d_softmax(x, y):
    de_dnet = x
    y_flat = y.flatten()
    de_dnet[np.arange(y_flat.shape[0]), y_flat] = -(1-de_dnet[np.arange(y_flat.shape[0]), y_flat])
    de_dnet = -de_dnet
    return de_dnet


## Layer Class

In [82]:
class Layer:
	def __init__(self, prev_nodes: int, num_nodes: int, activation_func):
		self.weights = np.random.standard_normal((prev_nodes, num_nodes))
		self.weights = np.r_[np.zeros((1, num_nodes)), self.weights]
		self.num_nodes = num_nodes
		self.activation_func = activation_func
		
	def get_params(self):
		return self.weights.size

	def forward(self, inputs: np.array):
		self.last_input = np.c_[np.ones((inputs.shape[0], 1)), inputs]
		self.last_unactivated = self.last_input @ self.weights
		if self.activation_func == ActivationFunction.SIGMOID:
	 		self.last_output = Activations.sigmoid(self.last_unactivated)
		elif self.activation_func == ActivationFunction.RELU:
			self.last_output = Activations.relu(self.last_unactivated)
		elif self.activation_func == ActivationFunction.LINEAR:
			self.last_output = Activations.linear(self.last_unactivated)
		elif self.activation_func == ActivationFunction.SOFTMAX:
			self.last_output = Activations.softmax(self.last_unactivated)
		return self.last_output
	
	def backpropagate(self, delta: np.ndarray, lr: float, y: np.ndarray = None):
		if y is not None:
			if self.activation_func == ActivationFunction.SIGMOID:
				de_dnet = Activations.d_sigmoid(self.last_unactivated) * (y-self.last_output)
			elif self.activation_func == ActivationFunction.RELU:
				de_dnet = Activations.d_relu(self.last_unactivated) * (y-self.last_output)
			elif self.activation_func == ActivationFunction.LINEAR:
				de_dnet = Activations.d_linear(self.last_unactivated) * (y-self.last_output)
			elif self.activation_func == ActivationFunction.SOFTMAX:
				de_dnet = Activations.d_softmax(self.last_output, y)
		else:
			if self.activation_func == ActivationFunction.SIGMOID:
				de_dnet = delta * Activations.d_sigmoid(self.last_unactivated)
			elif self.activation_func == ActivationFunction.RELU:
				de_dnet = delta * Activations.d_relu(self.last_unactivated)
			elif self.activation_func == ActivationFunction.LINEAR:
				de_dnet = delta * Activations.d_linear(self.last_unactivated)
		curr_derivative = de_dnet.T@self.last_input
		grad_w = (de_dnet@self.weights.T)
		grad_w = grad_w[:,1:]
		self.weights += lr*curr_derivative.T
		return grad_w

	def display_table(self):
		print(tabulate([["Weights", self.weights], ["Activation", self.activation_func]], tablefmt='pretty'))

## Graph Class

In [83]:
class Graph:
	def __init__(self, input_count, n_layers, n_neurons, activation_funcs, lr, err_thresh, batch_size, max_iter=10000, print_per_iter=1000):
		self.layers = []
		self.lr = lr
		self.err_thresh = err_thresh
		self.max_iter = max_iter
		self.batch_size = batch_size
		self.n_layer = n_layers
		self.print_per_iter = print_per_iter
		for i in range(n_layers):
			activation_func = activation_funcs[i]
			if (n_neurons[i] is not None):
				n_neuron = n_neurons[i]
			else: 
				n_neuron = 3
			if (i>0):
				self.layers.append(Layer(self.layers[i-1].num_nodes, n_neuron, activation_func))
			else:
				self.layers.append(Layer(input_count, n_neuron, activation_func))
			self.output_activation = self.layers[-1].activation_func

	def __str__(self):
		return "Graph with {} layers".format(len(self.layers))
	
	def add_layer(self, layer: Layer):
		self.layers.append(layer)

	def predict(self, x: np.ndarray):
		h = x
		for i,l in enumerate(self.layers):
			h = l.forward(h)
		return h
			
	def loss(self, yhat: np.ndarray, y: np.ndarray):
			return np.sum(np.square(y-yhat))/2

	def softmax_loss(self, yhat: np.ndarray, y: np.ndarray):
			return -np.log(yhat[np.arange(yhat.shape[0]), y.flatten()]).sum()/yhat.shape[0]

	def train(self, x_train: np.ndarray, y_train: np.ndarray):
		count_iter = 0
		while 1:
			err = 0
			for batch in range((x_train.shape[0]//self.batch_size)+(1 if x_train.shape[0]%self.batch_size > 0 else 0)):
				lower = batch*self.batch_size
				upper = (batch+1)*self.batch_size
				batch_x = x_train[lower:upper,:]
				batch_y = y_train[lower:upper,:]

				yhat = self.predict(batch_x)

				if (self.output_activation == ActivationFunction.SOFTMAX):
					err += self.softmax_loss(yhat, batch_y)
				else:
					err += self.loss(yhat, batch_y)


				delta = self.layers[-1].backpropagate(None, self.lr, batch_y)
				for i, l in enumerate(self.layers[-2::-1]):
					delta = l.backpropagate(delta, self.lr)
			count_iter+= 1
			if count_iter % self.print_per_iter == 0:
				print(f'Iteration {count_iter}: error {err:.7f}')
			if count_iter >= self.max_iter:
				print("Reached max iteration")
				print(f'Ended at iteration: {count_iter}')
				break
			if err <= self.err_thresh:
				print("Reached error below threshold")
				print(f'Ended at iteration: {count_iter}')
		print(f'Final error: {err}')
		
	def display_table(self):
		trainable = 0
		for i,l in enumerate(self.layers):
			print(f'Layer {i}:')
			l.display_table()
			print()
			trainable += l.get_params()
		print(tabulate([["Number of hidden layers: ", self.n_layer - 1], ['Total trainable params: ', trainable]], tablefmt='pretty'))


## Main Function

In [84]:
from sklearn import datasets

iris = datasets.load_iris()
x, y = iris.data, iris.target
graf = Graph(len(iris.feature_names), 2, [3, len(iris.target_names)], ['sigmoid', 'softmax'], 1e-2, 2e-2,50)
graf.train(x, y.reshape(-1,1))

Iteration 1000: error 1.7438625
Iteration 2000: error 1.6974770
Iteration 3000: error 1.6943078
Iteration 4000: error 1.6934626
Iteration 5000: error 1.6922446
Iteration 6000: error 1.6686615
Iteration 7000: error 0.3037951
Iteration 8000: error 0.2557990
Iteration 9000: error 0.2462458
Iteration 10000: error 0.2458087
Reached max iteration
Ended at iteration: 10000
Final error: 0.24580868785853754


In [85]:
yhat_1 = graf.predict(x)
yhat = np.argmax(yhat_1, axis=1)
print(iris.target_names[yhat])

['setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'virginica'
 'versicolor' 'virginica' 'versicolor' 'virginica' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'virginica'
 'virginica' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versic

In [86]:
graf.display_table()

Layer 0:
+------------+--------------------------------------------+
|  Weights   | [[  0.4957162   11.98815757  -0.43667087]  |
|            |  [  0.57388781   9.68238803  -0.92292776]  |
|            |  [  3.3749871    9.46174545  -1.8931384 ]  |
|            |  [ -3.94710594 -16.29427717   0.82437936]  |
|            |  [ -5.18296249 -13.96861802  -1.37772225]] |
| Activation |                  sigmoid                   |
+------------+--------------------------------------------+

Layer 1:
+------------+-----------------------------------------+
|  Weights   | [[-5.16465071  0.99555748  4.16909323]  |
|            |  [13.95311871 -2.48011453 -9.66990733]  |
|            |  [ 2.40827307  4.53929817 -6.9025371 ]  |
|            |  [-2.51977239 -0.02532598  0.02571495]] |
| Activation |                 softmax                 |
+------------+-----------------------------------------+

+--------------------------+----+
| Number of hidden layers: | 1  |
| Total trainable params:  | 27 |